In [ ]:
import numpy as np
import torch
from datasets import load_dataset
from uncertainty.utils import utils
import hashlib
import random
import gc
from transformers import AutoTokenizer, AutoModelForCausalLM, StoppingCriteria, StoppingCriteriaList
import torch._dynamo

In [2]:
class StopWordsCriteria(StoppingCriteria):
    def __init__(self, stop_strings, tokenizer, input_ids):
        super().__init__()
        self.stop_strings = stop_strings
        self.tokenizer = tokenizer
        self.input_len = input_ids.shape[1]

    def __call__(self, input_ids, scores, **kwargs):
        # Decode only the newly generated tokens
        decoded = self.tokenizer.batch_decode(input_ids[:, self.input_len:], skip_special_tokens=True)
        for text in decoded:
            if any(stop_string in text for stop_string in self.stop_strings):
                print("STOPPING CRITERIA EVOKED")
                return True  # Stop generation
        return False

In [ ]:
seed = 42
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
from transformers import set_seed

set_seed(seed) 

In [3]:
model_id = "google/gemma-2-2b"
tokenizer = AutoTokenizer.from_pretrained(model_id, device_map='auto', token_type_ids=None, clean_up_tokenization_spaces=False)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype=torch.bfloat16, device_map='auto')
token_limit = 8192
    
def batch_predict(self, prompts, temperature, return_latent=False, batch_size=10):
    """Generate answers for a batch of prompts and return text, log-likelihoods, and embeddings if requested."""
    import torch
    device = "cuda" if torch.cuda.is_available() else "cpu"
    results = []

    for batch_start in range(0, len(prompts), batch_size):
        batch_prompts = prompts[batch_start:batch_start + batch_size]

       # Tokenize batch prompts
        encoded = self.tokenizer(
                batch_prompts,
                padding=True,
                truncation=True,
                max_length=self.token_limit - self.max_new_tokens,
                return_tensors="pt"
        ).to(device)

        # Define your stop strings
        # stop_strings = ["\n\n", "Question:", "Context:", "Now, provide", "Use exactly", "Evaluation"]
        stop_strings = []

        # Build stopping criteria
        stop_criteria = StoppingCriteriaList([
                StopWordsCriteria(stop_strings, self.tokenizer, encoded["input_ids"])
            ])

        allowed_keys = {"input_ids", "attention_mask"}
        safe_encoded = {k: v for k, v in encoded.items() if k in allowed_keys}

        # Now call generate with stopping_criteria
        with torch.no_grad():
            outputs = self.model.generate(
                    **safe_encoded,
                    max_new_tokens=self.max_new_tokens,
                    return_dict_in_generate=True,
                    output_scores=True,
                    output_hidden_states=True,
                    temperature=temperature,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id,
                    stopping_criteria=stop_criteria,
                    top_p = 0.9,
                    top_k = 50
                )
            
            eos_token_id = self.tokenizer.eos_token_id
            for i, sequence in enumerate(outputs.sequences):
                if eos_token_id in sequence:
                    print(f"[EOS Detected] Sample {i} includes eos_token_id ({eos_token_id})")
                    # Optional: where in the sequence
                    eos_positions = (sequence == eos_token_id).nonzero(as_tuple=True)[0].tolist()
                    print(f" → EOS token found at indices: {eos_positions}")

                else:
                    print(f"[EOS Not Found] Sample {i} did not include eos_token_id")

            # ---- FIXED SECTION ----
            # last layer → shape (batch, seq_len, hidden_size)
            
            hidden_states      = outputs.hidden_states[0][-1][0,0]
            last_token_embedding = hidden_states.cpu()
            
            for i, prompt in enumerate(batch_prompts):
                full_answer   = self.tokenizer.decode(outputs.sequences[i], skip_special_tokens=True)
                sliced_answer = full_answer[len(prompt):].strip()

                # Prompt / generation lengths
                token_stop_index = self.tokenizer(full_answer, return_tensors="pt")["input_ids"].shape[1]
                n_input_token    = encoded["input_ids"][i].ne(self.tokenizer.pad_token_id).sum().item()
                n_generated      = token_stop_index - n_input_token or 1

                # -----------------------

                # Log-likelihoods for generated tokens
                transition_scores = self.model.compute_transition_scores(
                    outputs.sequences, outputs.scores, normalize_logits=True
                )
                log_likelihoods = [score.item() for score in transition_scores[i][:n_generated]]

                if return_latent:
                    results.append((sliced_answer, log_likelihoods, (last_token_embedding, None, None)))
                else:
                    results.append((sliced_answer, log_likelihoods, None))

        return results

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]